In [80]:
import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.ipython_useSVG = True  # set this to False if you want PNGs instead of SVGs

In [81]:
DATA_FOLDER: Path = Path("../data/raw/SAMPL.csv")
data = pd.read_csv(DATA_FOLDER)
indane = data.loc[data['iupac'] == 'indane']

def molecule_get_smile(df: pd.DataFrame, name: str) -> str: 
    # Return smile 
    assert df.columns.__contains__('iupac'), "Dataframe must contain columns 'iupac' and 'smiles'."
    return df.loc[df['iupac'] == name]['smiles'].iloc[0]


In [ ]:
def get_atom_features(atom: Chem.rdchem.Atom):
    """Extract features for a single atom"""
    return [
        atom.GetAtomicNum(),            # Atomic number
        atom.GetDegree(),               # Number of bonds
        atom.GetFormalCharge(),         # Charge
        atom.GetHybridization().real,   # Hybridization
        atom.GetIsAromatic(),           # Aromaticity
        atom.GetTotalNumHs(),           # Number of hydrogens
    ]

In [79]:
mol = Chem.MolFromSmiles(molecule_get_smile(data, 'benzene'))
for atom in mol.GetAtoms(): 
    print(get_atom_features(atom))
    break

[6, 2, 0, 3, True, 1]
